<a href="https://colab.research.google.com/github/yuyu990116/transformers_tutorials/blob/main/P2_summarization(seq2seq%2CGLM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#GLM只有编码器，借助注意力掩码实现编码器解码器的效果

In [2]:
!pip install --upgrade pip
from google.colab import drive
drive.mount("/content/drive")
import os
os.chdir("/content/drive/MyDrive/nlp")
!pip install datases
!pip install -U transformers
!pip install -U accelerate #Trainer用
!pip install rouge-chinese  #评估用
!pip install sentencepiece #T5模型需要
from datasets import load_dataset,Dataset
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM,DataCollatorForSeq2Seq,Seq2SeqTrainer,Seq2SeqTrainingArguments,pipeline
import numpy as np
import torch
from rouge_chinese import Rouge

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ERROR: Could not find a version that satisfies the requirement datases (from versions: none)
ERROR: No matching distribution found for datases


In [25]:
datasets=load_dataset("BelleGroup/train_1M_CN")
# ds = ds.train_test_split(test_size=0.2, shuffle=True)
# ds = ds.train_test_split(100, seed=42)

#数据集太大了，裁剪一下
from datasets import Dataset,DatasetDict
train = Dataset.from_dict(datasets["train"][:50])
test = Dataset.from_dict(datasets["train"][50:70])
dataset_dict = DatasetDict({'train': train,'test':test})
# GLM不太好评估，所以不需要留validation数据，同时也不需要在训练的时候进行eval

In [26]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 50
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 20
    })
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/glm-2b", trust_remote_code=True)
model = AutoModelForSeq2SeqLM.from_pretrained("THUDM/glm-2b", trust_remote_code=True)

In [5]:
model=model.half()   #转为半精度，加速，减少占用资源

In [27]:
# 把output全部mask掉
def data_process(examples):
  contents = [i+j+tokenizer.mask_token for i,j in zip(examples["instruction"],examples["input"])]
  inputs = tokenizer(contents, max_length=256, truncation=True, padding="max_length", return_tensors="pt")
  inputs = tokenizer.build_inputs_for_generation(inputs, targets=examples['output'], padding=True, max_gen_length=64)
  return inputs

In [28]:
ds = dataset_dict.map(data_process,batched=True,remove_columns=datasets["train"].column_names)
ds

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'position_ids', 'attention_mask', 'labels'],
        num_rows: 50
    })
    test: Dataset({
        features: ['input_ids', 'position_ids', 'attention_mask', 'labels'],
        num_rows: 20
    })
})

In [34]:
dataset_dict["train"][0]

{'instruction': '判断给定的文章是否符合语法规则。如果不符合，请提供修改建议。\n下面是一篇文章的开头: "为了探讨这个主题，本文将提供一系列数据和实例，以证明这一观点。"\n',
 'input': '',
 'output': '这个开头符合语法规则。'}

In [29]:
print(tokenizer.decode(ds["train"][0]["input_ids"]))
print("labels:,",ds["train"][0]["labels"])
print("positions:",ds["train"][0]["position_ids"])
#["position_ids"]是两部分的positionids，第一部分是全局的，第二部分是由每个mask开始的局部的position

[CLS]判断给定的文章是否符合语法规则。如果不符合，请提供修改建议。
下面是一篇文章的开头: "为了探讨这个主题，本文将提供一系列数据和实例，以证明这一观点。"
[MASK]<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|

In [ ]:
# args = Seq2SeqTrainingArguments(
#     output_dir="./summary_glm",
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=8,
#     gradient_accumulation_steps=8,
#     logging_steps=8,
#     num_train_epochs=1
# )

In [31]:
args = Seq2SeqTrainingArguments(
    output_dir="./summary_glm",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=8,
    save_steps=8,
    num_train_epochs=1
)#

In [ ]:
trainer = Seq2SeqTrainer(
    args=args,
    model=model,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
)
# GLM不太好评估，所以不需要留validation数据，同时也不需要在训练的时候进行eval
trainer.train()

In [ ]:
model = model.eval()
input_text = dataset_dict["test"][-1]["instruction"] + dataset_dict["test"][-1]["input"]
inputs = tokenizer(input_text + tokenizer.mask_token, return_tensors="pt")
inputs = tokenizer.build_inputs_for_generation(inputs, max_gen_length=64)
inputs = inputs.to("cuda")
output = model.generate(**inputs, max_new_tokens=64, eos_token_id=tokenizer.eop_token_id, do_sample=True)
# GLM的eos_token_id是tokenizer.eop_token_id   eop:end of piece 每个mask是一个piece
tokenizer.decode(output[0].tolist())

In [39]:
def predict_for_test():
  predict = []
  with torch.inference_mode():
    for t in dataset_dict["test"]:
      inputs = tokenizer(t["instruction"] + t["input"] + tokenizer.mask_token, return_tensors="pt")
      inputs = tokenizer.build_inputs_for_generation(inputs, max_gen_length=64)
      inputs = inputs.to("cuda")
      output = model.generate(**inputs, max_new_tokens=64, eos_token_id=tokenizer.eop_token_id)#这里如果采用do_sample会对评估产生影响
      predict.append(tokenizer.decode(output[0].tolist()).split("<|startofpiece|>")[1].replace("<|endofpiece|>", "").strip())
      #predict只要被mask掉的部分（也就是预测的output部分）
      print("curID:", len(predict))
  return predict

In [ ]:
preds = predict_for_test()
preds

In [ ]:
rouge = Rouge()

docode_preds = ["".join(p) for p in preds]
decode_labels = ["".join(l) for l in dataset_dict["test"]["output"]]
scores = rouge.get_scores(docode_preds, decode_labels, avg=True)
{
    "rouge-1": scores["rouge-1"]["f"],
    "rouge-2": scores["rouge-2"]["f"],
    "rouge-l": scores["rouge-l"]["f"],
}